In [1]:
import pandas_datareader.data as web #to collect data
import datetime as dt #to specify start and end dates

# import yfinance as yf

import eventstudy as es
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as stats


import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.rolling import RollingOLS

from patsy import dmatrices
from tqdm.notebook import tqdm
tqdm.pandas()

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Data Reading

In [2]:
car = pd.read_csv("Director Level CAR_on_appointment v010225.csv", index_col = 0)

C:\Users\SHIVAM\AppData\Local\Temp\ipykernel_12608\1846434778.py:1: DtypeWarning: Columns (22,23,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  car = pd.read_csv("Director Level CAR_on_appointment v010225.csv", index_col = 0)


In [3]:
car

,Symbol,Company,AsOnDate,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Appointment Date,Member of Civil Services,Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Cessation Date,Cessation Reason,Brief Profile,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Occupation,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Tenure Valid till,PrevLastServed,Indep,cumsum_U,cumsum_Y,cumsum_N,OpBalUnc,OpBalIndep,OpBalNonIndep,OpBalTotal,CloBalUnc,CloBalIndep,CloBalNonIndep,CloBalTotal,YOE,YOEmax,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB,Rookie,IsIndep,IsNonIndep,IsRookie,IsNonRookie,IsCeoMD,IsChairman,IsPromoterClassification,IsPromoterBoard,IsPromoter,IsDualityChairmanMD,IsFamilyManaged,IsFamilyChairman,IsFamilyChairmanAndCEO,IsRookieIndep,IsRookieNonIndep,IsNonRookieIndep,IsNonRookieNonIndep,IsFemale,Age,TenureInYearsIndep,TenureInYearsTotal,IsFirstTermIndep,IsRetires5y,IsTermLimitRetirement,IsDefaultTerm,AsOnYear,IsTurnOver,HasRetires5y,HasTermLimitRetirement,IsTurnOverIndep,CompanyName,ProwessCode,Entity type,Entity type code,Incorporation year,Industry group,Industry group code,NIC name,NIC code,Ownership group code,Ownership group,Age group,Head office address,ACP,pct,RF,RMRF,MF,SMB,HML,OLS120_intercept,OLS120_RMRF,OLS120_SMB,OLS120_HML,OLS120_r_squared,OLS120_adjusted_r_squared,OLS120_f_p_value,120CAR3,120CAR5,120CAR7,120CAR11,OLS150_intercept,OLS150_RMRF,OLS150_SMB,OLS150_HML,OLS150_r_squared,OLS150_adjusted_r_squared,OLS150_f_p_value,150CAR3,150CAR5,150CAR7,150CAR11,OLS180_intercept,OLS180_RMRF,OLS180_SMB,OLS180_HML,OLS180_r_squared,OLS180_adjusted_r_squared,OLS180_f_p_value,180CAR3,180CAR5,180CAR7,180CAR11
0,3MINDIA,3M INDIA LTD.,2006-03-31,P00033272,MR.,SHANKARANARAYANA,VISHWANATHA RAO,BANAWARA,1960-05-14,M,INDIA,2002-07-24,NONE,GRADUATE: B.COM.,"POST GRADUATE: M.B.A.-FINANCE,BANGALORE UNIVER...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,FINANCE,LEADERSHIP/MAN...",NaN,NaN,"MR. B.V. SHANKARANARAYANA RAO, 57, HOLDS A BAC...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,1,0,0,1346.0,0,0,1346.0,1711.0,0,0,1711.0,1346.0,1346.0,(),"(('3M INDIA LTD.', 'U'),)","(('3M INDIA LTD.', 'U'),)",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,64.0,0.0,3.687671,0,0,0,0,2006,0,0,0,0,3M India Ltd.,36277.0,Public Ltd.,1.020302e+10,1987.0,"Plastic furniture, floorings & miscellaneous i...",1.010120e+14,Manufacture of other plastics products n.e.c.,22209.0,2.020200e+10,Private (Foreign),Between 1986 and 1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3MINDIA,3M INDIA LTD.,2006-03-31,P00000249,LATE MR.,BALAJI RAO,JAGANNATH RAO,DOVETON,1939-12-15,M,INDIA,2001-12-26,NONE,"GRADUATE: ASSOCIATE MEMBER,INDIAN INSTITUTION ...","GRADUATE: B.E.-MECHANICAL,UNIVERSITY OF MADRAS...","POST GRADUATE: M.E.-MECHANICAL,UNIVERSITY OF M...",POST GRADUATE: P.G.-ADVANCED MANAGEMENT PROGRA...,POST GRADUATE: P.G.DIPLOMA-INDUSTRIAL ENGINEER...,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaN,NaN,"MR. D.J. BALAJI RAO, 73, HOLDS A B.E DEGREE

In [4]:
car["AsOnDate"] = pd.to_datetime(car["AsOnDate"], format = "%Y-%m-%d")

In [5]:
car.columns.to_list()

['Symbol',
 'Company',
 'AsOnDate',
 'Person Code',
 'Director Salutation',
 'Director First Name',
 'Director Middle Name',
 'Director Surname',
 'Date of Birth',
 'Gender',
 'Nationality',
 'Appointment Date',
 'Member of Civil Services',
 'Education1',
 'Education2',
 'Education3',
 'Education4',
 'Education5',
 'Education6',
 'Education7',
 'Education8',
 'Education9',
 'Education10',
 'Skills/Competencies',
 'Cessation Date',
 'Cessation Reason',
 'Brief Profile',
 'Promoter Director (Yes/No)',
 'Position on Board',
 'Independent (Yes/No)',
 'Occupation',
 'Other Directorship 1',
 'Other Directorship 2',
 'Other Directorship 3',
 'Other Directorship 4',
 'Other Directorship 5',
 'Other Directorship 6',
 'Other Directorship 7',
 'Other Directorship 8',
 'Other Directorship 9',
 'Other Directorship 10',
 'Other Directorship 11',
 'Other Directorship 12',
 'Other Directorship 13',
 'Other Directorship 14',
 'Other Directorship 15',
 'Tenure Valid till',
 'PrevLastServed',
 'Indep',
 

### Only govt firms only

In [6]:
car = car.loc[ car["Ownership group code"] >= 20000000000, :].copy()	

# 2013 - 2020

In [7]:
carS1 = car.loc[ (car["AsOnDate"] < "2021-01-01") & (car["AsOnDate"] > "2012-12-31"), : ].copy()
carS1 = carS1.drop_duplicates( subset = ["Person Code", "Appointment Date"] ).reset_index(drop = True)
carS1

,Symbol,Company,AsOnDate,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Appointment Date,Member of Civil Services,Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Cessation Date,Cessation Reason,Brief Profile,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Occupation,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Tenure Valid till,PrevLastServed,Indep,cumsum_U,cumsum_Y,cumsum_N,OpBalUnc,OpBalIndep,OpBalNonIndep,OpBalTotal,CloBalUnc,CloBalIndep,CloBalNonIndep,CloBalTotal,YOE,YOEmax,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB,Rookie,IsIndep,IsNonIndep,IsRookie,IsNonRookie,IsCeoMD,IsChairman,IsPromoterClassification,IsPromoterBoard,IsPromoter,IsDualityChairmanMD,IsFamilyManaged,IsFamilyChairman,IsFamilyChairmanAndCEO,IsRookieIndep,IsRookieNonIndep,IsNonRookieIndep,IsNonRookieNonIndep,IsFemale,Age,TenureInYearsIndep,TenureInYearsTotal,IsFirstTermIndep,IsRetires5y,IsTermLimitRetirement,IsDefaultTerm,AsOnYear,IsTurnOver,HasRetires5y,HasTermLimitRetirement,IsTurnOverIndep,CompanyName,ProwessCode,Entity type,Entity type code,Incorporation year,Industry group,Industry group code,NIC name,NIC code,Ownership group code,Ownership group,Age group,Head office address,ACP,pct,RF,RMRF,MF,SMB,HML,OLS120_intercept,OLS120_RMRF,OLS120_SMB,OLS120_HML,OLS120_r_squared,OLS120_adjusted_r_squared,OLS120_f_p_value,120CAR3,120CAR5,120CAR7,120CAR11,OLS150_intercept,OLS150_RMRF,OLS150_SMB,OLS150_HML,OLS150_r_squared,OLS150_adjusted_r_squared,OLS150_f_p_value,150CAR3,150CAR5,150CAR7,150CAR11,OLS180_intercept,OLS180_RMRF,OLS180_SMB,OLS180_HML,OLS180_r_squared,OLS180_adjusted_r_squared,OLS180_f_p_value,180CAR3,180CAR5,180CAR7,180CAR11
0,20MICRONS,20 MICRONS LTD.,2013-03-31,P00027662,MR.,DIPTIMOY,DINESHCHANDRA,BHATTACHARJEE,1955-12-09,M,NaN,2005-04-29,NaN,GRADUATE: B.SC.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-03-02,NOMINATION ENDED,"MR.D.D.BHATTACHARJEE, IDBI-NOMINEE DIRECTOR HO...",NO,"NOMINEE DIRECTOR, EXECUTIVE DIRECTOR",NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-03-31,N,4,0,1,2892.0,0,0,2892.0,2892.0,0,365,3257.0,2893.0,2893.0,"(('20MICRONS', 'U'),)","(('20 MICRONS LTD.', 'N'),)","(('20 MICRONS LTD.', 'N'),)","(('20MICRONS', 'U'),)","(('20MICRONS', 'U'),)",1,0,0,1,1,0,0,1,0,0,1,1,0,0,1,1,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,68.0,0.0,7.923288,0,0,0,0,2013,0,0,0,0,20 Microns Ltd.,11.0,Public Ltd.,1.020302e+10,1987.0,Minerals,1.010203e+14,Other mining and quarrying,8.0,2.010104e+10,20 Microns Group,Between 1986 and 1990,"347, G I D C Industrial Estate, Waghodia, Vad...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20MICRONS,20 MICRONS LTD.,2013-03-31,P00027633,MR.,RAMKISHAN,AMIRCHAND,DEVIDAYAL,1950-12-08,M,INDIA,2007-10-13,NONE,POST GRADUATE: M.B.A.,"POST GRADUATE: M.COM.,MAHARAJA SAYAJIRAO UNIVE...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaN,NaN,MR.RAM DEVIDAYAL HAS A MASTER'S DEGREE IN COMM...,NO,NON-EXECUTIVE DIRECTOR,YES,NaN,20 MICRONS LTD.,BANCO PRODUCTS (INDIA) LTD.,MUNJAL AUTO INDUSTRIES LTD.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201

## 120 Days

### 120 CAR 3

In [8]:
group1 = carS1.loc[ carS1["IsRookieIndep"] == 1, ["120CAR3"]].dropna()
group2 = carS1.loc[ carS1["IsNonRookieIndep"] == 1, ["120CAR3"]].dropna()

group3 = carS1.loc[ carS1["IsRookieNonIndep"] == 1, ["120CAR3"]].dropna()
group4 = carS1.loc[ carS1["IsNonRookieNonIndep"] == 1, ["120CAR3"]].dropna()

In [9]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [-0.5572365]  P Value: [0.5773817]
Mean1: [-0.00435789]  Mean2: [-0.00279733]  Diff: [-0.00156056]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-1.35347901]  P Value: [0.17601252]
Mean1: [-0.00357618]  Mean2: [0.00632842]  Diff: [-0.0099046]


In [10]:
group1.describe()

,120CAR3
count,3869.000000
mean,-0.004358
std,0.125627
min,-5.546831
25%,-0.037638
50%,-0.004855
75%,0.031328
max,0.631866


### 120 CAR 5

In [11]:
group1 = carS1.loc[ carS1["IsRookieIndep"] == 1, ["120CAR5"]].dropna()
group2 = carS1.loc[ carS1["IsNonRookieIndep"] == 1, ["120CAR5"]].dropna()

group3 = carS1.loc[ carS1["IsRookieNonIndep"] == 1, ["120CAR5"]].dropna()
group4 = carS1.loc[ carS1["IsNonRookieNonIndep"] == 1, ["120CAR5"]].dropna()

In [12]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [-1.00864212]  P Value: [0.3131772]
Mean1: [-0.0080704]  Mean2: [-0.00385372]  Diff: [-0.00421668]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-1.41735771]  P Value: [0.15648844]
Mean1: [-0.0033886]  Mean2: [0.00800908]  Diff: [-0.01139767]


### 120 CAR 7

In [13]:
group1 = carS1.loc[ carS1["IsRookieIndep"] == 1, ["120CAR7"]].dropna()
group2 = carS1.loc[ carS1["IsNonRookieIndep"] == 1, ["120CAR7"]].dropna()

group3 = carS1.loc[ carS1["IsRookieNonIndep"] == 1, ["120CAR7"]].dropna()
group4 = carS1.loc[ carS1["IsNonRookieNonIndep"] == 1, ["120CAR7"]].dropna()

In [14]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [-0.80863956]  P Value: [0.41874689]
Mean1: [-0.01073275]  Mean2: [-0.00683068]  Diff: [-0.00390207]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-1.02095768]  P Value: [0.30735692]
Mean1: [-0.00409281]  Mean2: [0.00440205]  Diff: [-0.00849485]


### 120 CAR 11

In [15]:
group1 = carS1.loc[ carS1["IsRookieIndep"] == 1, ["120CAR11"]].dropna()
group2 = carS1.loc[ carS1["IsNonRookieIndep"] == 1, ["120CAR11"]].dropna()

group3 = carS1.loc[ carS1["IsRookieNonIndep"] == 1, ["120CAR11"]].dropna()
group4 = carS1.loc[ carS1["IsNonRookieNonIndep"] == 1, ["120CAR11"]].dropna()

In [16]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [-1.22452235]  P Value: [0.22079858]
Mean1: [-0.01567976]  Mean2: [-0.00451768]  Diff: [-0.01116208]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-0.77687221]  P Value: [0.43728845]
Mean1: [-0.00702294]  Mean2: [8.55203161e-05]  Diff: [-0.00710847]


## 150 Days

### 150 CAR 3

In [17]:
group1 = carS1.loc[ carS1["IsRookieIndep"] == 1, ["150CAR3"]].dropna()
group2 = carS1.loc[ carS1["IsNonRookieIndep"] == 1, ["150CAR3"]].dropna()

group3 = carS1.loc[ carS1["IsRookieNonIndep"] == 1, ["150CAR3"]].dropna()
group4 = carS1.loc[ carS1["IsNonRookieNonIndep"] == 1, ["150CAR3"]].dropna()

In [18]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [-1.03065599]  P Value: [0.30273448]
Mean1: [-0.00451897]  Mean2: [-0.00191092]  Diff: [-0.00260805]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-1.45235531]  P Value: [0.14651978]
Mean1: [-0.00341992]  Mean2: [0.00734245]  Diff: [-0.01076237]


### 150 CAR 5

In [19]:
group1 = carS1.loc[ carS1["IsRookieIndep"] == 1, ["150CAR5"]].dropna()
group2 = carS1.loc[ carS1["IsNonRookieIndep"] == 1, ["150CAR5"]].dropna()

group3 = carS1.loc[ carS1["IsRookieNonIndep"] == 1, ["150CAR5"]].dropna()
group4 = carS1.loc[ carS1["IsNonRookieNonIndep"] == 1, ["150CAR5"]].dropna()

In [20]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [-1.1828698]  P Value: [0.2368969]
Mean1: [-0.00813131]  Mean2: [-0.00347307]  Diff: [-0.00465825]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-1.5621613]  P Value: [0.11836623]
Mean1: [-0.00308192]  Mean2: [0.00962106]  Diff: [-0.01270298]


### 150 CAR 7

In [21]:
group1 = carS1.loc[ carS1["IsRookieIndep"] == 1, ["150CAR7"]].dropna()
group2 = carS1.loc[ carS1["IsNonRookieIndep"] == 1, ["150CAR7"]].dropna()

group3 = carS1.loc[ carS1["IsRookieNonIndep"] == 1, ["150CAR7"]].dropna()
group4 = carS1.loc[ carS1["IsNonRookieNonIndep"] == 1, ["150CAR7"]].dropna()

In [22]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [-1.15821401]  P Value: [0.24681219]
Mean1: [-0.01127506]  Mean2: [-0.00592881]  Diff: [-0.00534625]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-1.21704234]  P Value: [0.2236889]
Mean1: [-0.00385829]  Mean2: [0.00632162]  Diff: [-0.0101799]


### 150 CAR 11

In [23]:
group1 = carS1.loc[ carS1["IsRookieIndep"] == 1, ["150CAR11"]].dropna()
group2 = carS1.loc[ carS1["IsNonRookieIndep"] == 1, ["150CAR11"]].dropna()

group3 = carS1.loc[ carS1["IsRookieNonIndep"] == 1, ["150CAR11"]].dropna()
group4 = carS1.loc[ carS1["IsNonRookieNonIndep"] == 1, ["150CAR11"]].dropna()

In [24]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [-1.59484411]  P Value: [0.11079677]
Mean1: [-0.01544897]  Mean2: [-0.00143943]  Diff: [-0.01400954]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-1.19024648]  P Value: [0.23403993]
Mean1: [-0.00654712]  Mean2: [0.00424704]  Diff: [-0.01079416]


## 180 Days

### 180 CAR 3

In [25]:
group1 = carS1.loc[ carS1["IsRookieIndep"] == 1, ["180CAR3"]].dropna()
group2 = carS1.loc[ carS1["IsNonRookieIndep"] == 1, ["180CAR3"]].dropna()

group3 = carS1.loc[ carS1["IsRookieNonIndep"] == 1, ["180CAR3"]].dropna()
group4 = carS1.loc[ carS1["IsNonRookieNonIndep"] == 1, ["180CAR3"]].dropna()

In [26]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [0.78110482]  P Value: [0.43478038]
Mean1: [-0.00490414]  Mean2: [-0.00958958]  Diff: [0.00468545]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-1.46036783]  P Value: [0.14430854]
Mean1: [-0.00360598]  Mean2: [0.00734952]  Diff: [-0.01095549]


### 180 CAR 5

In [27]:
group1 = carS1.loc[ carS1["IsRookieIndep"] == 1, ["180CAR5"]].dropna()
group2 = carS1.loc[ carS1["IsNonRookieIndep"] == 1, ["180CAR5"]].dropna()

group3 = carS1.loc[ carS1["IsRookieNonIndep"] == 1, ["180CAR5"]].dropna()
group4 = carS1.loc[ carS1["IsNonRookieNonIndep"] == 1, ["180CAR5"]].dropna()

In [28]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [0.50931363]  P Value: [0.61055541]
Mean1: [-0.00934006]  Mean2: [-0.01370667]  Diff: [0.00436661]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-1.65166496]  P Value: [0.09871998]
Mean1: [-0.00352327]  Mean2: [0.01008947]  Diff: [-0.01361275]


### 180 CAR 7

In [29]:
group1 = carS1.loc[ carS1["IsRookieIndep"] == 1, ["180CAR7"]].dropna()
group2 = carS1.loc[ carS1["IsNonRookieIndep"] == 1, ["180CAR7"]].dropna()

group3 = carS1.loc[ carS1["IsRookieNonIndep"] == 1, ["180CAR7"]].dropna()
group4 = carS1.loc[ carS1["IsNonRookieNonIndep"] == 1, ["180CAR7"]].dropna()

In [30]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [0.43510532]  P Value: [0.66350393]
Mean1: [-0.01248649]  Mean2: [-0.01637772]  Diff: [0.00389124]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-1.32165541]  P Value: [0.18639134]
Mean1: [-0.0041819]  Mean2: [0.00704541]  Diff: [-0.01122731]


### 180 CAR 11

In [31]:
group1 = carS1.loc[ carS1["IsRookieIndep"] == 1, ["180CAR11"]].dropna()
group2 = carS1.loc[ carS1["IsNonRookieIndep"] == 1, ["180CAR11"]].dropna()

group3 = carS1.loc[ carS1["IsRookieNonIndep"] == 1, ["180CAR11"]].dropna()
group4 = carS1.loc[ carS1["IsNonRookieNonIndep"] == 1, ["180CAR11"]].dropna()

In [32]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [-0.03042303]  P Value: [0.97573089]
Mean1: [-0.01725299]  Mean2: [-0.01682808]  Diff: [-0.00042491]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-1.27551195]  P Value: [0.20222602]
Mean1: [-0.00672345]  Mean2: [0.00486935]  Diff: [-0.01159279]


# 2013 - 2024

In [33]:
carS2 = car.loc[ (car["AsOnDate"] < "2024-01-01") & (car["AsOnDate"] > "2012-12-31"), : ].copy()
carS2 = carS2.drop_duplicates( subset = ["Person Code", "Appointment Date"] ).reset_index(drop = True)
carS2

,Symbol,Company,AsOnDate,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Appointment Date,Member of Civil Services,Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Cessation Date,Cessation Reason,Brief Profile,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Occupation,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Tenure Valid till,PrevLastServed,Indep,cumsum_U,cumsum_Y,cumsum_N,OpBalUnc,OpBalIndep,OpBalNonIndep,OpBalTotal,CloBalUnc,CloBalIndep,CloBalNonIndep,CloBalTotal,YOE,YOEmax,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB,Rookie,IsIndep,IsNonIndep,IsRookie,IsNonRookie,IsCeoMD,IsChairman,IsPromoterClassification,IsPromoterBoard,IsPromoter,IsDualityChairmanMD,IsFamilyManaged,IsFamilyChairman,IsFamilyChairmanAndCEO,IsRookieIndep,IsRookieNonIndep,IsNonRookieIndep,IsNonRookieNonIndep,IsFemale,Age,TenureInYearsIndep,TenureInYearsTotal,IsFirstTermIndep,IsRetires5y,IsTermLimitRetirement,IsDefaultTerm,AsOnYear,IsTurnOver,HasRetires5y,HasTermLimitRetirement,IsTurnOverIndep,CompanyName,ProwessCode,Entity type,Entity type code,Incorporation year,Industry group,Industry group code,NIC name,NIC code,Ownership group code,Ownership group,Age group,Head office address,ACP,pct,RF,RMRF,MF,SMB,HML,OLS120_intercept,OLS120_RMRF,OLS120_SMB,OLS120_HML,OLS120_r_squared,OLS120_adjusted_r_squared,OLS120_f_p_value,120CAR3,120CAR5,120CAR7,120CAR11,OLS150_intercept,OLS150_RMRF,OLS150_SMB,OLS150_HML,OLS150_r_squared,OLS150_adjusted_r_squared,OLS150_f_p_value,150CAR3,150CAR5,150CAR7,150CAR11,OLS180_intercept,OLS180_RMRF,OLS180_SMB,OLS180_HML,OLS180_r_squared,OLS180_adjusted_r_squared,OLS180_f_p_value,180CAR3,180CAR5,180CAR7,180CAR11
0,20MICRONS,20 MICRONS LTD.,2013-03-31,P00027662,MR.,DIPTIMOY,DINESHCHANDRA,BHATTACHARJEE,1955-12-09,M,NaN,2005-04-29,NaN,GRADUATE: B.SC.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-03-02,NOMINATION ENDED,"MR.D.D.BHATTACHARJEE, IDBI-NOMINEE DIRECTOR HO...",NO,"NOMINEE DIRECTOR, EXECUTIVE DIRECTOR",NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-03-31,N,4,0,1,2892.0,0,0,2892.0,2892.0,0,365,3257.0,2893.0,2893.0,"(('20MICRONS', 'U'),)","(('20 MICRONS LTD.', 'N'),)","(('20 MICRONS LTD.', 'N'),)","(('20MICRONS', 'U'),)","(('20MICRONS', 'U'),)",1,0,0,1,1,0,0,1,0,0,1,1,0,0,1,1,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,68.0,0.0,7.923288,0,0,0,0,2013,0,0,0,0,20 Microns Ltd.,11.0,Public Ltd.,1.020302e+10,1987.0,Minerals,1.010203e+14,Other mining and quarrying,8.0,2.010104e+10,20 Microns Group,Between 1986 and 1990,"347, G I D C Industrial Estate, Waghodia, Vad...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20MICRONS,20 MICRONS LTD.,2013-03-31,P00027633,MR.,RAMKISHAN,AMIRCHAND,DEVIDAYAL,1950-12-08,M,INDIA,2007-10-13,NONE,POST GRADUATE: M.B.A.,"POST GRADUATE: M.COM.,MAHARAJA SAYAJIRAO UNIVE...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaN,NaN,MR.RAM DEVIDAYAL HAS A MASTER'S DEGREE IN COMM...,NO,NON-EXECUTIVE DIRECTOR,YES,NaN,20 MICRONS LTD.,BANCO PRODUCTS (INDIA) LTD.,MUNJAL AUTO INDUSTRIES LTD.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201

## 120 Days

### 120 CAR 3

In [34]:
group1 = carS2.loc[ carS2["IsRookieIndep"] == 1, ["120CAR3"]].dropna()
group2 = carS2.loc[ carS2["IsNonRookieIndep"] == 1, ["120CAR3"]].dropna()

group3 = carS2.loc[ carS2["IsRookieNonIndep"] == 1, ["120CAR3"]].dropna()
group4 = carS2.loc[ carS2["IsNonRookieNonIndep"] == 1, ["120CAR3"]].dropna()

In [35]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [0.295947]  P Value: [0.76727663]
Mean1: [-0.00395907]  Mean2: [-0.00464609]  Diff: [0.00068701]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-1.112002]  P Value: [0.26621559]
Mean1: [-0.00315564]  Mean2: [0.00357125]  Diff: [-0.00672689]


### 120 CAR 5

In [36]:
group1 = carS2.loc[ carS2["IsRookieIndep"] == 1, ["120CAR5"]].dropna()
group2 = carS2.loc[ carS2["IsNonRookieIndep"] == 1, ["120CAR5"]].dropna()

group3 = carS2.loc[ carS2["IsRookieNonIndep"] == 1, ["120CAR5"]].dropna()
group4 = carS2.loc[ carS2["IsNonRookieNonIndep"] == 1, ["120CAR5"]].dropna()

In [37]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [-0.16406643]  P Value: [0.86968229]
Mean1: [-0.00755014]  Mean2: [-0.00696067]  Diff: [-0.00058946]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-1.41006518]  P Value: [0.15860906]
Mean1: [-0.00404493]  Mean2: [0.00531255]  Diff: [-0.00935748]


### 120 CAR 7

In [38]:
group1 = carS2.loc[ carS2["IsRookieIndep"] == 1, ["120CAR7"]].dropna()
group2 = carS2.loc[ carS2["IsNonRookieIndep"] == 1, ["120CAR7"]].dropna()

group3 = carS2.loc[ carS2["IsRookieNonIndep"] == 1, ["120CAR7"]].dropna()
group4 = carS2.loc[ carS2["IsNonRookieNonIndep"] == 1, ["120CAR7"]].dropna()

In [39]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [0.1286077]  P Value: [0.89767093]
Mean1: [-0.01026602]  Mean2: [-0.01082839]  Diff: [0.00056237]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-1.11763941]  P Value: [0.26379282]
Mean1: [-0.00565454]  Mean2: [0.00208022]  Diff: [-0.00773477]


### 120 CAR 11

In [40]:
group1 = carS2.loc[ carS2["IsRookieIndep"] == 1, ["120CAR11"]].dropna()
group2 = carS2.loc[ carS2["IsNonRookieIndep"] == 1, ["120CAR11"]].dropna()

group3 = carS2.loc[ carS2["IsRookieNonIndep"] == 1, ["120CAR11"]].dropna()
group4 = carS2.loc[ carS2["IsNonRookieNonIndep"] == 1, ["120CAR11"]].dropna()

In [41]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [-0.45267729]  P Value: [0.65079432]
Mean1: [-0.01492278]  Mean2: [-0.01133807]  Diff: [-0.00358471]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-0.77547527]  P Value: [0.43810207]
Mean1: [-0.00940475]  Mean2: [-0.00341533]  Diff: [-0.00598942]


## 150 Days

### 150 CAR 3

In [42]:
group1 = carS2.loc[ carS2["IsRookieIndep"] == 1, ["150CAR3"]].dropna()
group2 = carS2.loc[ carS2["IsNonRookieIndep"] == 1, ["150CAR3"]].dropna()

group3 = carS2.loc[ carS2["IsRookieNonIndep"] == 1, ["150CAR3"]].dropna()
group4 = carS2.loc[ carS2["IsNonRookieNonIndep"] == 1, ["150CAR3"]].dropna()

In [43]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [-0.09558961]  P Value: [0.92384841]
Mean1: [-0.00394835]  Mean2: [-0.00374846]  Diff: [-0.00019988]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-1.23679556]  P Value: [0.21625093]
Mean1: [-0.00275599]  Mean2: [0.00477078]  Diff: [-0.00752677]


### 150 CAR 5

In [44]:
group1 = carS2.loc[ carS2["IsRookieIndep"] == 1, ["150CAR5"]].dropna()
group2 = carS2.loc[ carS2["IsNonRookieIndep"] == 1, ["150CAR5"]].dropna()

group3 = carS2.loc[ carS2["IsRookieNonIndep"] == 1, ["150CAR5"]].dropna()
group4 = carS2.loc[ carS2["IsNonRookieNonIndep"] == 1, ["150CAR5"]].dropna()

In [45]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [-0.37702844]  P Value: [0.70616098]
Mean1: [-0.00752186]  Mean2: [-0.00625817]  Diff: [-0.00126369]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-1.56620027]  P Value: [0.11739539]
Mean1: [-0.00336906]  Mean2: [0.00707933]  Diff: [-0.01044839]


### 150 CAR 7

In [46]:
group1 = carS2.loc[ carS2["IsRookieIndep"] == 1, ["150CAR7"]].dropna()
group2 = carS2.loc[ carS2["IsNonRookieIndep"] == 1, ["150CAR7"]].dropna()

group3 = carS2.loc[ carS2["IsRookieNonIndep"] == 1, ["150CAR7"]].dropna()
group4 = carS2.loc[ carS2["IsNonRookieNonIndep"] == 1, ["150CAR7"]].dropna()

In [47]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [-0.24268222]  P Value: [0.80825698]
Mean1: [-0.0106039]  Mean2: [-0.00960922]  Diff: [-0.00099468]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-1.31579561]  P Value: [0.18832741]
Mean1: [-0.0048338]  Mean2: [0.00424337]  Diff: [-0.00907716]


### 150 CAR 11

In [48]:
group1 = carS2.loc[ carS2["IsRookieIndep"] == 1, ["150CAR11"]].dropna()
group2 = carS2.loc[ carS2["IsNonRookieIndep"] == 1, ["150CAR11"]].dropna()

group3 = carS2.loc[ carS2["IsRookieNonIndep"] == 1, ["150CAR11"]].dropna()
group4 = carS2.loc[ carS2["IsNonRookieNonIndep"] == 1, ["150CAR11"]].dropna()

In [49]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [-0.83144166]  P Value: [0.4057517]
Mean1: [-0.01461306]  Mean2: [-0.00833369]  Diff: [-0.00627936]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-1.18944619]  P Value: [0.234336]
Mean1: [-0.00810816]  Mean2: [0.00087379]  Diff: [-0.00898195]


## 180 Days

### 180 CAR 3

In [50]:
group1 = carS2.loc[ carS2["IsRookieIndep"] == 1, ["180CAR3"]].dropna()
group2 = carS2.loc[ carS2["IsNonRookieIndep"] == 1, ["180CAR3"]].dropna()

group3 = carS2.loc[ carS2["IsRookieNonIndep"] == 1, ["180CAR3"]].dropna()
group4 = carS2.loc[ carS2["IsNonRookieNonIndep"] == 1, ["180CAR3"]].dropna()

In [51]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [1.14970217]  P Value: [0.25031465]
Mean1: [-0.00425757]  Mean2: [-0.00982218]  Diff: [0.00556461]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-1.25140982]  P Value: [0.210875]
Mean1: [-0.00275781]  Mean2: [0.00492168]  Diff: [-0.0076795]


### 180 CAR 5

In [52]:
group1 = carS2.loc[ carS2["IsRookieIndep"] == 1, ["180CAR5"]].dropna()
group2 = carS2.loc[ carS2["IsNonRookieIndep"] == 1, ["180CAR5"]].dropna()

group3 = carS2.loc[ carS2["IsRookieNonIndep"] == 1, ["180CAR5"]].dropna()
group4 = carS2.loc[ carS2["IsNonRookieNonIndep"] == 1, ["180CAR5"]].dropna()

In [53]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [0.82073585]  P Value: [0.41182993]
Mean1: [-0.00851005]  Mean2: [-0.01420339]  Diff: [0.00569333]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-1.65889359]  P Value: [0.09723139]
Mean1: [-0.00358338]  Mean2: [0.00758981]  Diff: [-0.01117319]


### 180 CAR 7

In [54]:
group1 = carS2.loc[ carS2["IsRookieIndep"] == 1, ["180CAR7"]].dropna()
group2 = carS2.loc[ carS2["IsNonRookieIndep"] == 1, ["180CAR7"]].dropna()

group3 = carS2.loc[ carS2["IsRookieNonIndep"] == 1, ["180CAR7"]].dropna()
group4 = carS2.loc[ carS2["IsNonRookieNonIndep"] == 1, ["180CAR7"]].dropna()

In [55]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [0.82630643]  P Value: [0.40866188]
Mean1: [-0.01158054]  Mean2: [-0.01762357]  Diff: [0.00604303]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-1.50282992]  P Value: [0.13297298]
Mean1: [-0.0054166]  Mean2: [0.00507678]  Diff: [-0.01049337]


### 180 CAR 11

In [56]:
group1 = carS2.loc[ carS2["IsRookieIndep"] == 1, ["180CAR11"]].dropna()
group2 = carS2.loc[ carS2["IsNonRookieIndep"] == 1, ["180CAR11"]].dropna()

group3 = carS2.loc[ carS2["IsRookieNonIndep"] == 1, ["180CAR11"]].dropna()
group4 = carS2.loc[ carS2["IsNonRookieNonIndep"] == 1, ["180CAR11"]].dropna()

In [57]:
t_stat1, p_value1 = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)
t_stat2, p_value2 = stats.ttest_ind(group3, group4, equal_var=False)  # Welch’s t-test (default)


print("ROOKIE INDEP VS NON ROOKIE INDEP")
print("T Statistic:", t_stat1, " P Value:",p_value1)
print("Mean1:", group1.mean().values, " Mean2:", group2.mean().values, " Diff:", group1.mean().values - group2.mean().values)
print()

print("ROOKIE NON INDEP VS NON ROOKIE NON INDEP")
print("T Statistic:", t_stat2, " P Value:",p_value2)
print("Mean1:", group3.mean().values, " Mean2:", group4.mean().values, " Diff:", group3.mean().values - group4.mean().values)

ROOKIE INDEP VS NON ROOKIE INDEP
T Statistic: [0.36532428]  P Value: [0.71488225]
Mean1: [-0.01607871]  Mean2: [-0.02024755]  Diff: [0.00416884]

ROOKIE NON INDEP VS NON ROOKIE NON INDEP
T Statistic: [-1.47316655]  P Value: [0.14078389]
Mean1: [-0.00948337]  Mean2: [0.00178249]  Diff: [-0.01126585]
